In [2]:
import json
import requests
# TODO
# why would you update a root?
# How do you clone a scenario
# How do you create a whole new scenario based on a new branch
# Set up routines
# Set up studies
# Access results
# get all the scenarios where a specific agent branch is used

### Helper Functions

In [170]:
with open('../secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']
    # API_KEY = json.load(file)['LOCAL_API_KEY'] # For work in dev environment

headers = {'X_API_KEY': API_KEY, 'Content-Type': 'application/json'}

In [171]:
URL_BASE = 'https://api.sedaro.com'
# URL_BASE = 'http://localhost:80' # For work in dev environment
URL_Stems = {
    'WORKSPACES': URL_BASE + '/workspaces/',
    'REPOSITORIES': URL_BASE + '/models/repositories/',
    'BRANCHES': URL_BASE + '/models/branches/'
}

In [172]:
def get_all(object_type_url):
    q = requests.get(object_type_url, headers=headers)
    print('API URL: ', object_type_url)
    return [entry for entry in q.json()]

In [173]:
def get_item_by_name(object_type_url, name):
    q = requests.get(object_type_url, headers=headers)
    print('API URL: ', object_type_url)
    results = [entry for entry in q.json() if entry['name'] == name]
    if len(results) > 0:
        return results[0]
    else:
        return None

In [174]:
def get_item_by_id(object_type_url, id):
    url = object_type_url + id
    q = requests.get(url, headers=headers)
    print('API URL: ', url)
    if type(q.json()) is dict:
        return q.json()
    else:
        return None

In [175]:
# Also create block
def update_item(object_type_url, id, payload):
    item_id = id
    if object_type_url == URL_Stems['BRANCHES']:
        item_id = id + '/template'
    url = object_type_url + item_id
    q = requests.patch(url, headers=headers, data=payload)
    print('API URL: ', url)
    return q.json()

In [246]:
# When you create a block, you are technically updating/patching the branch
# def create_block(object_type_url, id, payload):
#     return update_item(object_type_url, id, payload)

In [176]:
# repos need a special method to get them by name
def get_repo_by_name(workspace_id, repo_name):
    ws = get_item_by_id(URL_Stems['WORKSPACES'], workspace_id)
    list_of_repos = []
    for repository in ws['repositories']:
        if repository['name'] == repo_name:
            list_of_repos.append(repository)
    # Or using a Python list comprehension
    # list_of_repos = [repository for repository in workspace['repositories'] if repository['name'] == repo_name]
    if len(list_of_repos) > 0:
        return list_of_repos[0]
    else:
        return None

In [200]:
def get_branch_by_name(repo_id, branch_name):
    rp = get_item_by_id(URL_Stems['REPOSITORIES'], repo_id)
    list_of_branches = []
    for branch in rp['branches']:
        if branch['name'] == branch_name:
            list_of_branches.append(branch)
    # Or using a Python list comprehension
    # list_of_branches = [branch for branch in rp['branches'] if branch['name'] == branch_name]
    if len(list_of_branches) > 0:
        return list_of_branches[0]
    else:
        return None

In [178]:
def create_repo(object_type_url, payload):
    q = requests.post(object_type_url, headers=headers, data=payload)
    print('API URL: ', object_type_url)
    return q.json()

In [179]:
def create_branch(object_type_url, id, payload):
    url = object_type_url + id
    q = requests.post(url, headers=headers, data=payload)
    print('API URL: ', url)
    return q.json()

In [238]:
def get_block_instances(branch_id, block_type):
    branch = get_item_by_id(URL_Stems['BRANCHES'], branch_id)
    if 'error' not in branch:
        block_ids = branch['data']['index'][block_type]
        blocks = {}
        for block_id in block_ids:
            block = branch['data']['blocks'][block_id]
            blocks[block['id']]=block
        if len(block_ids) > 0:
            return {'UIDs': block_ids, 'blocks': blocks}
        else:
            return {'UIDs': None, 'blocks': None}
    else:
        return branch

In [241]:
def get_block_instance_by_name(branch_id, block_type, name):
    block_results = get_block_instances(branch_id, block_type)
    print(block_results)
    if 'error' not in block_results:
        blocks = []
        if block_results['UIDs'] is not None:
            for id in block_results['UIDs']:
                b = block_results['blocks'][id]
                if b['name'] == name:
                    blocks.append(b)
        if len(blocks) > 0:
            return blocks[0]
        else:
            return None
    else:
        return block_results

In [249]:
def print_neat(dictionary_for_viewing):
    print(json.dumps(dictionary_for_viewing, indent=4, sort_keys=True))

### Workspaces

In [180]:
workspace_from_name = get_item_by_name(URL_Stems['WORKSPACES'], "LC Demos")
workspace_id = workspace_from_name['id']
# Watchout, workspaces can have the same name
workspace_id

API URL:  https://api.sedaro.com/workspaces/


'PLbmFrM26h37tNj6XjVvQD'

In [182]:
# Get workspace by id
workspace = get_item_by_id(URL_Stems['WORKSPACES'], workspace_id)
workspace

API URL:  https://api.sedaro.com/workspaces/PLbmFrM26h37tNj6XjVvQD


{'dateCreated': '2024-02-22T21:00:03.024Z',
 'dateModified': '2024-04-05T13:08:00.850Z',
 'defaultRole': 'PLbmFtH7Hv6tjtVm7CR4YH',
 'description': '',
 'id': 'PLbmFrM26h37tNj6XjVvQD',
 'isActive': True,
 'license': {'dateCreated': '2024-02-22T21:00:05.452Z',
  'dateExpires': '2024-02-29T21:00:05.452Z',
  'dateModified': '2024-02-22T21:00:05.452Z',
  'id': 'PLbmFtHHC7y6LYTql6nxy3',
  'isActive': True,
  'workspace': 'PLbmFrM26h37tNj6XjVvQD'},
 'members': [{'dateCreated': '2023-04-26T18:54:35.200Z',
   'dateModified': '2024-03-30T02:56:41.416Z',
   'email': 'lael.cox@sedarotech.com',
   'externalIdentityProvider': None,
   'firstName': 'Lael',
   'id': 'NTyuhTQSoD0PhFWUTCRiF',
   'incomplete': False,
   'isFlagged': False,
   'isVerified': True,
   'lastName': 'Cox',
   'organization': 'Sedaro',
   'organizationCountryCode': 'US',
   'roles': ['PLbmFtHFDvd4r2p6xWL2W6'],
   'userCountryCode': 'US',
   'workspaces': ['PLSd3LprQLXmJ4M99B8Rb3',
    'PLbmFrM26h37tNj6XjVvQD',
    'NTz2giOUaiZQ

In [183]:
# Rename a workspace
workspace_data = {'id': 'PLbmFrM26h37tNj6XjVvQD', 'name': 'LC Demos'}
update_item(URL_Stems['WORKSPACES'], 'PLbmFrM26h37tNj6XjVvQD', json.dumps(workspace_data))
# returns the workspace

API URL:  https://api.sedaro.com/workspaces/PLbmFrM26h37tNj6XjVvQD


{'dateCreated': '2024-02-22T21:00:03.024Z',
 'dateModified': '2024-04-05T13:11:34.629Z',
 'defaultRole': 'PLbmFtH7Hv6tjtVm7CR4YH',
 'description': '',
 'id': 'PLbmFrM26h37tNj6XjVvQD',
 'isActive': True,
 'license': {'dateCreated': '2024-02-22T21:00:05.452Z',
  'dateExpires': '2024-02-29T21:00:05.452Z',
  'dateModified': '2024-02-22T21:00:05.452Z',
  'id': 'PLbmFtHHC7y6LYTql6nxy3',
  'isActive': True,
  'workspace': 'PLbmFrM26h37tNj6XjVvQD'},
 'members': [{'dateCreated': '2023-04-26T18:54:35.200Z',
   'dateModified': '2024-03-30T02:56:41.416Z',
   'email': 'lael.cox@sedarotech.com',
   'externalIdentityProvider': None,
   'firstName': 'Lael',
   'id': 'NTyuhTQSoD0PhFWUTCRiF',
   'incomplete': False,
   'isFlagged': False,
   'isVerified': True,
   'lastName': 'Cox',
   'organization': 'Sedaro',
   'organizationCountryCode': 'US',
   'roles': ['PLbmFtHFDvd4r2p6xWL2W6'],
   'userCountryCode': 'US',
   'workspaces': ['PLSd3LprQLXmJ4M99B8Rb3',
    'PLbmFrM26h37tNj6XjVvQD',
    'NTz2giOUaiZQ

### Repos

In [204]:
# Create a Repository
repo_data = {'name': 'API_SC7', 'description': 'Test3',
                   'metamodelType': 'Spacecraft', 'workspace': workspace['id']}
new_repo = create_repo(URL_Stems['REPOSITORIES'], json.dumps(repo_data))
new_repo
# Returns the new repo

API URL:  https://api.sedaro.com/models/repositories/


{'branches': [{'createdBy': 'NTyuhTQSoD0PhFWUTCRiF',
   'data': {'_blockNames': ['ActivePointingMode',
     'AngularVelocitySensor',
     'Antenna',
     'AreaTarget',
     'AveragingAlgorithm',
     'Battery',
     'BatteryCell',
     'BatteryPack',
     'BodyFrameVector',
     'BodyInFovCondition',
     'BusRegulator',
     'CelestialTarget',
     'CelestialVector',
     'CircularFieldOfView',
     'CombinationalLogic',
     'Component',
     'ComponentToScalarCondition',
     'CompoundCondition',
     'Cooler',
     'CooperativeLineOfSightTransmitInterface',
     'DataBus',
     'DataInterface',
     'DataMode',
     'DataStorage',
     'DataType',
     'DirectMeasurementAttitudeInitializer',
     'DirectionSensor',
     'DynamicallyLoadedComponent',
     'EkfAlgorithm',
     'ElapsedTimeCondition',
     'FieldOfViewArticulationMode',
     'FiniteDifferenceOrbitInitializer',
     'FiniteStateMachine',
     'FixedSchedule',
     'FixedSurface',
     'FuelReservoir',
     'FullyRegDet

In [185]:
# To get all your repos across all workspaces you belong to (this can take a while)
all_repos = get_all(URL_Stems['REPOSITORIES'])
for r in all_repos:
    print('Workspace: ', r['workspace']['id'], 'Repo: ', r['name'])

API URL:  https://api.sedaro.com/models/repositories/
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  API_SC6
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  API_SC4
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  API_SC3
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  API_SC1
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  API_SC
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  Test12
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  Test11
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  Ground Station
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  Equatorial Ground Contact
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  LC_SimpleSat
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  drag-cannonball [import]
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  srp-cannonball [import]
Workspace:  PLbmFrM26h37tNj6XjVvQD Repo:  Orbit Validation
Workspace:  NTz2giOUaiZQ9R6LVXbw- Repo:  Simple
Workspace:  NTz2giOUaiZQ9R6LVXbw- Repo:  Simple2
Workspace:  NTz2giOUaiZQ9R6LVXbw- Repo:  LC Ground
Workspace:  NTz2giOUaiZQ9R6LVXbw- Repo:  [DEMO] Wildfire Scenarios [import]
Wor

In [186]:
# Get a list of repos in workspace
workspace['repositories']

[{'branches': ['PM4khPxTjCxXkJlLBsTT69'],
  'createdBy': 'NTyuhTQSoD0PhFWUTCRiF',
  'dateCreated': '2024-04-04T19:37:55.505Z',
  'dateModified': '2024-04-04T19:37:55.505Z',
  'description': 'Test3',
  'id': 'PM4khPv8m2tFH5JnvfdCCc',
  'lastModifiedBy': 'NTyuhTQSoD0PhFWUTCRiF',
  'metamodelType': 'Spacecraft',
  'name': 'API_SC4',
  'project': None,
  'repoType': 'AGENT_TEMPLATE',
  'versions': ['PM4khPxTjCxXkJlLBsTT69'],
  'workspace': 'PLbmFrM26h37tNj6XjVvQD'},
 {'branches': ['PM4jSxXnxxzNjZcJxwRPFy'],
  'createdBy': 'NTyuhTQSoD0PhFWUTCRiF',
  'dateCreated': '2024-04-04T18:33:17.572Z',
  'dateModified': '2024-04-04T18:33:17.572Z',
  'description': 'Test3',
  'id': 'PM4jSxVg4DcFmlgvDP9nm4',
  'lastModifiedBy': 'NTyuhTQSoD0PhFWUTCRiF',
  'metamodelType': 'Spacecraft',
  'name': 'API_SC3',
  'project': None,
  'repoType': 'AGENT_TEMPLATE',
  'versions': ['PM4jSxXnxxzNjZcJxwRPFy'],
  'workspace': 'PLbmFrM26h37tNj6XjVvQD'},
 {'branches': ['PM4jPM8Wkm5h4kRmkM4Qm2'],
  'createdBy': 'NTyuhTQS

In [251]:
# Getting a repo by name has its own function
repo = get_repo_by_name(workspace['id'],'API_SC')
repo

API URL:  https://api.sedaro.com/workspaces/PLbmFrM26h37tNj6XjVvQD


{'branches': ['PM47vRx4XcgfXX8LgNzwDK',
  'PM58tQXq7Gt49CBv7fy8sN',
  'PM4l3tg2Cqd9lmVcgC87Wr',
  'PM48rp2fJ5jdhfpqpzGGxC',
  'PM48q8qr2tmBnRGnknRkhJ',
  'PM47rRJnSKWwyH73gdLtYC'],
 'createdBy': 'NTyuhTQSoD0PhFWUTCRiF',
 'dateCreated': '2024-04-03T16:36:03.146Z',
 'dateModified': '2024-04-03T16:36:03.146Z',
 'description': 'Test3',
 'id': 'PM47rR7jsCHHdLM2RGLDtJ',
 'lastModifiedBy': 'NTyuhTQSoD0PhFWUTCRiF',
 'metamodelType': 'Spacecraft',
 'name': 'API_SC',
 'project': None,
 'repoType': 'AGENT_TEMPLATE',
 'versions': ['PM47vRx4XcgfXX8LgNzwDK',
  'PM58tQXq7Gt49CBv7fy8sN',
  'PM4l3tg2Cqd9lmVcgC87Wr',
  'PM48rp2fJ5jdhfpqpzGGxC',
  'PM48q8qr2tmBnRGnknRkhJ',
  'PM47rRJnSKWwyH73gdLtYC'],
 'workspace': 'PLbmFrM26h37tNj6XjVvQD'}

In [189]:
# Get a repo by id
repository_from_id = get_item_by_id(URL_Stems['REPOSITORIES'], repo['id'])
repository_from_id

API URL:  https://api.sedaro.com/models/repositories/PM47rR7jsCHHdLM2RGLDtJ


{'branches': [{'createdBy': 'NTyuhTQSoD0PhFWUTCRiF',
   'data': {'_blockNames': ['ActivePointingMode',
     'AngularVelocitySensor',
     'Antenna',
     'AreaTarget',
     'AveragingAlgorithm',
     'Battery',
     'BatteryCell',
     'BatteryPack',
     'BodyFrameVector',
     'BodyInFovCondition',
     'BusRegulator',
     'CelestialTarget',
     'CelestialVector',
     'CircularFieldOfView',
     'CombinationalLogic',
     'Component',
     'ComponentToScalarCondition',
     'CompoundCondition',
     'Cooler',
     'CooperativeLineOfSightTransmitInterface',
     'DataBus',
     'DataInterface',
     'DataMode',
     'DataStorage',
     'DataType',
     'DirectMeasurementAttitudeInitializer',
     'DirectionSensor',
     'DynamicallyLoadedComponent',
     'EkfAlgorithm',
     'ElapsedTimeCondition',
     'FieldOfViewArticulationMode',
     'FiniteDifferenceOrbitInitializer',
     'FiniteStateMachine',
     'FixedSchedule',
     'FixedSurface',
     'FuelReservoir',
     'FullyRegDet

In [140]:
repo

{'branches': ['PM47vRx4XcgfXX8LgNzwDK',
  'PM48rp2fJ5jdhfpqpzGGxC',
  'PM48q8qr2tmBnRGnknRkhJ',
  'PM47rRJnSKWwyH73gdLtYC'],
 'createdBy': 'NTyuhTQSoD0PhFWUTCRiF',
 'dateCreated': '2024-04-03T16:36:03.146Z',
 'dateModified': '2024-04-03T16:36:03.146Z',
 'description': 'Test3',
 'id': 'PM47rR7jsCHHdLM2RGLDtJ',
 'lastModifiedBy': 'NTyuhTQSoD0PhFWUTCRiF',
 'metamodelType': 'Spacecraft',
 'name': 'API_SC',
 'project': None,
 'repoType': 'AGENT_TEMPLATE',
 'versions': ['PM47vRx4XcgfXX8LgNzwDK',
  'PM48rp2fJ5jdhfpqpzGGxC',
  'PM48q8qr2tmBnRGnknRkhJ',
  'PM47rRJnSKWwyH73gdLtYC'],
 'workspace': 'PLbmFrM26h37tNj6XjVvQD'}

### Branches

In [190]:
# Get list of branches
repo['branches']

['PM4l3tg2Cqd9lmVcgC87Wr',
 'PM47vRx4XcgfXX8LgNzwDK',
 'PM48rp2fJ5jdhfpqpzGGxC',
 'PM48q8qr2tmBnRGnknRkhJ',
 'PM47rRJnSKWwyH73gdLtYC']

In [201]:
# Get a branch by name
branch = get_branch_by_name('PM47rR7jsCHHdLM2RGLDtJ','main')
branch

API URL:  https://api.sedaro.com/models/repositories/PM47rR7jsCHHdLM2RGLDtJ


{'createdBy': 'NTyuhTQSoD0PhFWUTCRiF',
 'data': {'_blockNames': ['ActivePointingMode',
   'AngularVelocitySensor',
   'Antenna',
   'AreaTarget',
   'AveragingAlgorithm',
   'Battery',
   'BatteryCell',
   'BatteryPack',
   'BodyFrameVector',
   'BodyInFovCondition',
   'BusRegulator',
   'CelestialTarget',
   'CelestialVector',
   'CircularFieldOfView',
   'CombinationalLogic',
   'Component',
   'ComponentToScalarCondition',
   'CompoundCondition',
   'Cooler',
   'CooperativeLineOfSightTransmitInterface',
   'DataBus',
   'DataInterface',
   'DataMode',
   'DataStorage',
   'DataType',
   'DirectMeasurementAttitudeInitializer',
   'DirectionSensor',
   'DynamicallyLoadedComponent',
   'EkfAlgorithm',
   'ElapsedTimeCondition',
   'FieldOfViewArticulationMode',
   'FiniteDifferenceOrbitInitializer',
   'FiniteStateMachine',
   'FixedSchedule',
   'FixedSurface',
   'FuelReservoir',
   'FullyRegDetPowerProcessor',
   'GenericAdAlgorithm',
   'GenericOdAlgorithm',
   'GpsAlgorithm',
  

In [202]:
# Get a branch by id
branch_from_id = get_item_by_id(URL_Stems['BRANCHES'], 'PM47vRx4XcgfXX8LgNzwDK')
branch_from_id

API URL:  https://api.sedaro.com/models/branches/PM47vRx4XcgfXX8LgNzwDK


{'createdBy': {'dateCreated': '2023-04-26T18:54:35.200Z',
  'dateModified': '2024-03-30T02:56:41.416Z',
  'email': 'lael.cox@sedarotech.com',
  'externalIdentityProvider': None,
  'firstName': 'Lael',
  'id': 'NTyuhTQSoD0PhFWUTCRiF',
  'incomplete': False,
  'isFlagged': False,
  'isVerified': True,
  'lastName': 'Cox',
  'organization': 'Sedaro',
  'organizationCountryCode': 'US',
  'roles': ['NTz2gzIRRXIC1vkE1PEtV',
   'PJskg3GfQFffDH2ZfspGRf',
   'PKCbxCmTsFPHpCKsbD7WCv',
   'NalsCPtTsegMUKS_J1XwV',
   'NXkVfgZSZXPbmqCFzfUGV',
   'PLbmFtHFDvd4r2p6xWL2W6',
   'PLcDcrxBtHtN8RlpKSWJfW',
   'PLSd3NhpgNY2KNVGXn7f5Q'],
  'userCountryCode': 'US',
  'workspaces': ['PLSd3LprQLXmJ4M99B8Rb3',
   'PLbmFrM26h37tNj6XjVvQD',
   'NTz2giOUaiZQ9R6LVXbw-',
   'PLcDcq37gxczq49GhBvhcy',
   'PJskg2QtjkLMj4kFzMDnWD',
   'NXkVfPrRKqcOutT-fBuO-',
   'PKCbxBkpxJdxwRbsZTFcDt',
   'NalsC8YTWPI4DxnG6z31k']},
 'data': {'_blockNames': ['ActivePointingMode',
   'AngularVelocitySensor',
   'Antenna',
   'AreaTarget

In [203]:
# Create a branch
branch_data = {'name': 'API Branch 8', 'description': 'Test Branch3',
                   'branchId': branch['id']}
create_branch(URL_Stems['BRANCHES'], branch['id'], payload=json.dumps(branch_data))

API URL:  https://api.sedaro.com/models/branches/PM47rRJnSKWwyH73gdLtYC


{'createdBy': {'dateCreated': '2023-04-26T18:54:35.200Z',
  'dateModified': '2024-03-30T02:56:41.416Z',
  'email': 'lael.cox@sedarotech.com',
  'externalIdentityProvider': None,
  'firstName': 'Lael',
  'id': 'NTyuhTQSoD0PhFWUTCRiF',
  'incomplete': False,
  'isFlagged': False,
  'isVerified': True,
  'lastName': 'Cox',
  'organization': 'Sedaro',
  'organizationCountryCode': 'US',
  'roles': ['NTz2gzIRRXIC1vkE1PEtV',
   'PJskg3GfQFffDH2ZfspGRf',
   'PKCbxCmTsFPHpCKsbD7WCv',
   'NalsCPtTsegMUKS_J1XwV',
   'NXkVfgZSZXPbmqCFzfUGV',
   'PLbmFtHFDvd4r2p6xWL2W6',
   'PLcDcrxBtHtN8RlpKSWJfW',
   'PLSd3NhpgNY2KNVGXn7f5Q'],
  'userCountryCode': 'US',
  'workspaces': ['PLSd3LprQLXmJ4M99B8Rb3',
   'PLbmFrM26h37tNj6XjVvQD',
   'NTz2giOUaiZQ9R6LVXbw-',
   'PLcDcq37gxczq49GhBvhcy',
   'PJskg2QtjkLMj4kFzMDnWD',
   'NXkVfPrRKqcOutT-fBuO-',
   'PKCbxBkpxJdxwRbsZTFcDt',
   'NalsC8YTWPI4DxnG6z31k']},
 'data': {'_blockNames': ['ActivePointingMode',
   'AngularVelocitySensor',
   'Antenna',
   'AreaTarget

### Working with Blocks

In [ ]:
# Pattern to create
url = URL_Stems['BRANCHES'] + id + '/template'

data = {'blocks': [{
    'id': '',  # optional
    'type': '',
    # 'property': 'value'
    }]}

q = requests.post(url, headers=headers, data=json.dumps(data))
print(json.dumps(q.json(), indent=2))

In [239]:
get_block_instances('PM47vRx4XcgfXX8LgNzwDK','BodyFrameVector')

API URL:  https://api.sedaro.com/models/branches/PM47vRx4XcgfXX8LgNzwDK


{'UIDs': ['N_dA5fnT5DwNn40QSKR4V',
  'N_dAvrMSdmYcjRnM1qKSV',
  'N_dB7gWR61wdN8AmaTK9F',
  'N_dBDWdUEiQQIB7eb91fV',
  'N_dBGp6Ri9NCGOtCqElJV',
  'N_dBJ7DT1HEtYaCVqbDzV',
  'PM48vdvmLTRC3NVPKHwmwn',
  'PM4gYblK4Rv7vKRlScch75'],
 'blocks': {'N_dA5fnT5DwNn40QSKR4V': {'definitionParams': {'vector': [1.0,
     0.0,
     0.0]},
   'definitionType': 'VECTOR',
   'disabled': False,
   'geocenterAngle': None,
   'heliocenterAngle': None,
   'id': 'N_dA5fnT5DwNn40QSKR4V',
   'name': '+X',
   'type': 'BodyFrameVector',
   'unitVector': [1.0, 0.0, 0.0]},
  'N_dAvrMSdmYcjRnM1qKSV': {'definitionParams': {'vector': [0.0, 1.0, 0.0]},
   'definitionType': 'VECTOR',
   'disabled': False,
   'geocenterAngle': None,
   'heliocenterAngle': None,
   'id': 'N_dAvrMSdmYcjRnM1qKSV',
   'name': '+Y',
   'type': 'BodyFrameVector',
   'unitVector': [0.0, 1.0, 0.0]},
  'N_dB7gWR61wdN8AmaTK9F': {'definitionParams': {'vector': [0.0, 0.0, 1.0]},
   'definitionType': 'VECTOR',
   'disabled': False,
   'geocenterAngle'

In [242]:
get_block_instance_by_name('PM47vRx4XcgfXX8LgNzwD', 'BodyFrameVector','+X')

API URL:  https://api.sedaro.com/models/branches/PM47vRx4XcgfXX8LgNzwD
{'error': {'code': 'ENDPOINT_NOT_FOUND', 'message': 'The requested endpoint does not exist or is not accessible with your current permissions.', 'status': 404}}


{'error': {'code': 'ENDPOINT_NOT_FOUND',
  'message': 'The requested endpoint does not exist or is not accessible with your current permissions.',
  'status': 404}}

### Other Useful Blocks

In [247]:
# To create a block, you are updating the branch
data = {'blocks': [{
            'name': 'VT6',
            'type': 'BodyFrameVector',
            'definitionType': 'SPHERICAL_ANGLES',
            'definitionParams': {
                'phi': 7,
                'theta': 2
            }
        }]}

update_item(URL_Stems['BRANCHES'], 'PM47vRx4XcgfXX8LgNzwDK', json.dumps(data))
# print(json.dumps(q.json(), indent=2))

API URL:  https://api.sedaro.com/models/branches/PM47vRx4XcgfXX8LgNzwDK/template


{'branch': {'createdBy': {'dateCreated': '2023-04-26T18:54:35.200Z',
   'dateModified': '2024-03-30T02:56:41.416Z',
   'email': 'lael.cox@sedarotech.com',
   'externalIdentityProvider': None,
   'firstName': 'Lael',
   'id': 'NTyuhTQSoD0PhFWUTCRiF',
   'incomplete': False,
   'isFlagged': False,
   'isVerified': True,
   'lastName': 'Cox',
   'organization': 'Sedaro',
   'organizationCountryCode': 'US',
   'roles': ['NTz2gzIRRXIC1vkE1PEtV',
    'PJskg3GfQFffDH2ZfspGRf',
    'PKCbxCmTsFPHpCKsbD7WCv',
    'NalsCPtTsegMUKS_J1XwV',
    'NXkVfgZSZXPbmqCFzfUGV',
    'PLbmFtHFDvd4r2p6xWL2W6',
    'PLcDcrxBtHtN8RlpKSWJfW',
    'PLSd3NhpgNY2KNVGXn7f5Q'],
   'userCountryCode': 'US',
   'workspaces': ['PLSd3LprQLXmJ4M99B8Rb3',
    'PLbmFrM26h37tNj6XjVvQD',
    'NTz2giOUaiZQ9R6LVXbw-',
    'PLcDcq37gxczq49GhBvhcy',
    'PJskg2QtjkLMj4kFzMDnWD',
    'NXkVfPrRKqcOutT-fBuO-',
    'PKCbxBkpxJdxwRbsZTFcDt',
    'NalsC8YTWPI4DxnG6z31k']},
  'data': {'_blockNames': ['ActivePointingMode',
    'AngularVelo

In [248]:
#Invoke ideal pointing mode, you are updating the branch
data = {'blocks': [{
    'type': "IdealOrbitalAttitudeDynamics"
}]}
update_item(URL_Stems['BRANCHES'], 'PM47vRx4XcgfXX8LgNzwDK', json.dumps(data))

# When you switch between pointing modes, it just overwrites the block
# These create and update actions return the whole branch

API URL:  https://api.sedaro.com/models/branches/PM47vRx4XcgfXX8LgNzwDK/template


{'branch': {'createdBy': {'dateCreated': '2023-04-26T18:54:35.200Z',
   'dateModified': '2024-03-30T02:56:41.416Z',
   'email': 'lael.cox@sedarotech.com',
   'externalIdentityProvider': None,
   'firstName': 'Lael',
   'id': 'NTyuhTQSoD0PhFWUTCRiF',
   'incomplete': False,
   'isFlagged': False,
   'isVerified': True,
   'lastName': 'Cox',
   'organization': 'Sedaro',
   'organizationCountryCode': 'US',
   'roles': ['NTz2gzIRRXIC1vkE1PEtV',
    'PJskg3GfQFffDH2ZfspGRf',
    'PKCbxCmTsFPHpCKsbD7WCv',
    'NalsCPtTsegMUKS_J1XwV',
    'NXkVfgZSZXPbmqCFzfUGV',
    'PLbmFtHFDvd4r2p6xWL2W6',
    'PLcDcrxBtHtN8RlpKSWJfW',
    'PLSd3NhpgNY2KNVGXn7f5Q'],
   'userCountryCode': 'US',
   'workspaces': ['PLSd3LprQLXmJ4M99B8Rb3',
    'PLbmFrM26h37tNj6XjVvQD',
    'NTz2giOUaiZQ9R6LVXbw-',
    'PLcDcq37gxczq49GhBvhcy',
    'PJskg2QtjkLMj4kFzMDnWD',
    'NXkVfPrRKqcOutT-fBuO-',
    'PKCbxBkpxJdxwRbsZTFcDt',
    'NalsC8YTWPI4DxnG6z31k']},
  'data': {'_blockNames': ['ActivePointingMode',
    'AngularVelo

{
    "_blockNames": [
        "ActivePointingMode",
        "AngularVelocitySensor",
        "Antenna",
        "AreaTarget",
        "AveragingAlgorithm",
        "Battery",
        "BatteryCell",
        "BatteryPack",
        "BodyFrameVector",
        "BodyInFovCondition",
        "BusRegulator",
        "CelestialTarget",
        "CelestialVector",
        "CircularFieldOfView",
        "CombinationalLogic",
        "Component",
        "ComponentToScalarCondition",
        "CompoundCondition",
        "Cooler",
        "CooperativeLineOfSightTransmitInterface",
        "DataBus",
        "DataInterface",
        "DataMode",
        "DataStorage",
        "DataType",
        "DirectMeasurementAttitudeInitializer",
        "DirectionSensor",
        "DynamicallyLoadedComponent",
        "EkfAlgorithm",
        "ElapsedTimeCondition",
        "FieldOfViewArticulationMode",
        "FiniteDifferenceOrbitInitializer",
        "FiniteStateMachine",
        "FixedSchedule",
        "Fi